<a href="https://colab.research.google.com/github/GerardoMunoz/SPARCV8over4/blob/main/Cos_Taylor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparing two Integers and a Floating-Point implementations of Cosine

The purpose of this code is to compare different approaches to computing the Taylor series expansion of cosine using fixed-point arithmetic.  
We will analyze two different integer implementations:  

1. **Direct multiplication of numerators and denominators separately**  
2. **Performing division at each step to control number growth**  

The objective is to determine which method is more suitable for 32-bit registers and evaluate their accuracy compared to floating-point calculations.  


In [1]:
import math

def taylor_cos(x, k=256, terms=10):
    # Constants
    pi = math.pi
    x_squared = x * x
    k_squared = k * k
    two_pi_squared = (2 * pi) ** 2

    # Initialize numerator and denominator for the first cycle
    numerator = k  # T_0 numerator (cos(x) starts with k)
    denominator = 1  # T_0 denominator (cos(x) starts with 1)
    acc_i1 = numerator // denominator
    acc_i2 = numerator // denominator
    acc_f = numerator / denominator
    term_i2 = numerator // denominator

    # Iterate over terms (each term will update numerator and denominator)
    for n in range(1, terms + 1):
        # Numerator calculation: T_n_numerator = T_(n-1)_numerator * (-x^2 * (2π)^2)
        numerator = numerator * (-x_squared * two_pi_squared)

        # Denominator calculation: T_n_denominator = T_(n-1)_denominator * (2n)(2n-1) * k^2
        denominator = denominator * (2 * n) * ((2 * n) - 1) * k_squared

        # Calculate the term T_n
        term_i1 = numerator // denominator  # Integer division to keep the result as an integer
        acc_i1 += term_i1

        term_i2 = term_i2*(-x_squared * two_pi_squared)//((2 * n) * ((2 * n) - 1) * k_squared)
        acc_i2 += term_i2

        term_f = numerator / denominator
        acc_f += term_f

        print(f"Cycle {n}: numerator = {numerator}, denominator = {denominator}, term_i1 = {term_i1}, acc_i1 = {acc_i1}, term_i2 = {term_i2}, acc_i2 = {acc_i2}, term_f = {term_f}, acc_f = {acc_f}")

    return acc_i2

# Test the function
k=2**8

x = k//4
result = taylor_cos(x,k=k)
print(f"{k}*cos({x}*2*pi/{k}) -> {result}")

x = k//2
result = taylor_cos(x,k=k)
print(f"{k}*cos({x}*2*pi/{k}) -> {result}")

x = k
result = taylor_cos(x,k=k)
print(f"{k}*cos({x}*2*pi/{k}) -> {result}")


Cycle 1: numerator = -41396121.2179067, denominator = 131072, term_i1 = -316.0, acc_i1 = -60.0, term_i2 = -316.0, acc_i2 = -60.0, term_f = -315.82734083485946, acc_f = -59.827340834859456
Cycle 2: numerator = 6693901765186.036, denominator = 103079215104, term_i1 = 64.0, acc_i1 = 4.0, term_i2 = 64.0, acc_i2 = 4.0, term_f = 64.93939402266828, acc_f = 5.112053187808826
Cycle 3: numerator = -1.0824280034859405e+18, denominator = 202661983231672320, term_i1 = -6.0, acc_i1 = -2.0, term_i2 = -6.0, acc_i2 = -2.0, term_f = -5.341051075418357, acc_f = -0.22899788760953133
Cycle 4: numerator = 1.750325032889091e+23, denominator = 743772721051969121157120, term_i1 = 0.0, acc_i1 = -2.0, term_i2 = 0.0, acc_i2 = -2.0, term_f = 0.23533063035889315, acc_f = 0.006332742749361825
Cycle 5: numerator = -2.8303385637583335e+28, denominator = 4386950014217566349173771468800, term_i1 = -1.0, acc_i1 = -3.0, term_i2 = -0.0, acc_i2 = -2.0, term_f = -0.0064517228475035136, acc_f = -0.00011898009814168865
Cycle 6

We can see that the first type of integer calculation generates numbers too large to be handled in 32-bit registers.  
However, the second method manages to keep values within the 32-bit range by performing division at each cycle.  

The error compared to floating-point results is as follows:  
- For a **90° angle**, the error is (2-0)/256
- For a **180° angle**, the error is (-258 - -256)/256
- For a **360° angle**, the error is (254 - 256)/256

This analysis suggests that performing division at each step provides a practical balance between accuracy and computational feasibility on 32-bit hardware.  


## Simplified code

In [23]:
import math
pi = math.pi

def kcos(x, k=256, terms=10):
    # Constants
    two_pi_squared = int((2 * pi) ** 2)
    x_squared = x * x
    k_squared = k * k
    num = x_squared * two_pi_squared
    num = -num

    term_i2 = k
    acc_i2 = term_i2
    # Iterate over terms
    for n in range(1, terms + 1):
        n2 = (2 * n)
        n2m1 = ((2 * n) - 1)
        fac2 = n2m1 * n2
        den = fac2 * k_squared
        term_i2 = term_i2 * num
        term_i2 = term_i2 // den
        acc_i2 += term_i2
        print(f"acc: {acc_i2}, term: {term_i2}, num: {num}, den: {den}")
    return acc_i2

# Test the function
k=256
x=64
print(kcos(x,k=k))


acc: -56, term: -312, num: -159744, den: 131072
acc: 7, term: 63, num: -159744, den: 786432
acc: 1, term: -6, num: -159744, den: 1966080
acc: 1, term: 0, num: -159744, den: 3670016
acc: 1, term: 0, num: -159744, den: 5898240
acc: 1, term: 0, num: -159744, den: 8650752
acc: 1, term: 0, num: -159744, den: 11927552
acc: 1, term: 0, num: -159744, den: 15728640
acc: 1, term: 0, num: -159744, den: 20054016
acc: 1, term: 0, num: -159744, den: 24903680
1


In [22]:
import math
pi = math.pi

def kcos(x, k=256, terms=10):
    # Constants
    two_pi_squared = int((2 * pi) ** 2)
    x_squared = x * x
    k_squared = k * k

    term_i2 = k
    acc_i2 = term_i2
    # Iterate over terms
    for n in range(1, terms + 1):
        term_i2 = term_i2*(-x_squared * two_pi_squared)//((2 * n) * ((2 * n) - 1) * k_squared)
        acc_i2 += term_i2
        print(f"acc: {acc_i2}, term: {term_i2}")
    return acc_i2

# Test the function
k=256
x=64
print(kcos(x,k=k))


acc: -56, term: -312
acc: 7, term: 63
acc: 1, term: -6
acc: 1, term: 0
acc: 1, term: 0
acc: 1, term: 0
acc: 1, term: 0
acc: 1, term: 0
acc: 1, term: 0
acc: 1, term: 0
1


In [4]:
import statistics
import math

errors=[]
for x in range(k):
    result = kcos(x,k=k)
    error=(k*math.cos(x/k*(2*pi))-result)/k
    errors.append(error)
    print(f"{k}*cos({x}*2*pi/{k}) -> {result}, error = {error}")

print("\nStatistics for errors:")
print(f"Max error: {max(errors)}")
print(f"Average error: {sum(errors) / len(errors)}")
print(f"Standard deviation of errors: {statistics.stdev(errors)}")


256*cos(0*2*pi/256) -> 256, error = 0.0
256*cos(1*2*pi/256) -> 255, error = 0.00360506869620425
256*cos(2*2*pi/256) -> 255, error = 0.002701706205172405
256*cos(3*2*pi/256) -> 255, error = 0.001196706678690207
256*cos(4*2*pi/256) -> 254, error = 0.0029972266721969287
256*cos(5*2*pi/256) -> 254, error = 0.00029203459870996706
256*cos(6*2*pi/256) -> 253, error = 0.0008952599647810144
256*cos(7*2*pi/256) -> 252, error = 0.0009026423889412216
256*cos(8*2*pi/256) -> 251, error = 0.0003165304032304306
256*cos(9*2*pi/256) -> 249, error = 0.00304588003852857
256*cos(10*2*pi/256) -> 248, error = 0.0012812531945439742
256*cos(11*2*pi/256) -> 246, error = 0.0028385657954398402
256*cos(12*2*pi/256) -> 245, error = -9.091426779117562e-05
256*cos(13*2*pi/256) -> 243, error = 0.00030943059303667475
256*cos(14*2*pi/256) -> 241, error = 0.0001378151830208063
256*cos(15*2*pi/256) -> 238, error = 0.0033052988347389567
256*cos(16*2*pi/256) -> 236, error = 0.0020045325112867385
256*cos(17*2*pi/256) -> 233,

```
MOV   R1, 64            !x
CALL  4
NOP
NOP
NOP
    MOV   R3, 39            ! KCOS: two_pi_squared
    SMUL  R2, R1, R1        ! x^2 = x * x
    MOV   R4, 65536         ! k^2 = 256^2
    MOV   R6, 256           ! term_i2 = k
    MOV   R5, R6            ! acc_i2 = term_i2
    SMUL   R10, R2, R3        ! x^2 * two_pi_squared
    NEG    R10, R10           ! -x^2 * two_pi_squared
        MOV   R7, 1               ! n = 1
        SUBcc   R0,R7, 10         ! LOOP.  if n > terms, exit
        BE   12
        NOP
        SLL    R9, R7, 1         ! 2 * n
        SUB    R8, R9, 1         ! (2 * n) - 1
        SMUL   R8, R8, R9        ! (2 * n) * ((2 * n) - 1)
        SMUL   R8, R8, R4         ! Multiply by k^2
        SMUL   R6, R6, R10        ! term_i2 *= numerator
        SDIV   R6, R6, R8         ! term_i2 /= denominator
        ADD    R5, R5, R6         ! acc_i2 += term_i2
        ADD    R7, R7, 1          ! n += 1
        BA    -11
        NOP
    RETL                      !END Return acc_i2 in R5

```